# Building a Multi-Agent System with `smolagents`

In this notebook, we will explore how to use the [`smolagents`](https://huggingface.co/docs/smolagents) library from Hugging Face to build a simple multi-agent system.

Our goal is to create an agent system that collaborates to put together a lecture outline.  

## What You’ll Learn  

By the end of this exercise, you will:  
- Understand the basics of multi-agent systems and how agents can coordinate to solve tasks.  
- Learn how to define agents using `smolagents` and equip them with reasoning capabilities.  
- Explore how agents can use tools like web search to gather relevant information.  
- Implement a system where agents iteratively refine a lecture outline based on user prompts.  

## Getting Started  

Run the cells below to install dependencies and initialize the multi-agent system. Let's dive in!  


In [1]:
!pip install smolagents[openai]==1.21.2 ddgs==9.5.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.4 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [2]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, OpenAIServerModel, LogLevel

## Setting Up Your OpenAI API Key  

This notebook requires an OpenAI API key to function. The key is used to authenticate requests when interacting with OpenAI's models.  

### How to Get an OpenAI API Key  

1. Go to the [OpenAI Platform](https://platform.openai.com/signup/) and sign up or log in.  
2. Navigate to the **API Keys** section in your account settings: [OpenAI API Keys](https://platform.openai.com/api-keys).  
3. Click **Create new secret key** and copy the generated key.  
4. Keep this key secure. Do not share it publicly or hardcode it in the notebook.  

Finally, execute the cell bellow and copy&paste the token into the provided field to configure the access of this notebook.

In [3]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("Enter your OPENAI API key: ")

Enter your OPENAI API key: ··········


## Defining the LLM used by the models
In this notebook, our multi-agent system will rely on a Large Language Model (LLM) to generate and refine content.

The `smolagents` library allows us to define agents that interact with an LLM to reason, plan, and use external tools.  

We will use OpenAI's `gpt-4o` model. However, you can modify the setup to use other models.

If you prefer an open-source alternative, you can modify the notebook to use a Hugging Face-hosted model or a locally deployed LLM like LLaMA, Mistral, or Falcon.


In [4]:
model = OpenAIServerModel("gpt-4o", max_tokens=8096)

## Defining Expert Agents with `CodeAgent`  

In this section, we will define two specialized expert agents using `smolagents.CodeAgent`. Each agent will have a distinct area of expertise and will contribute to building a structured lecture outline.  

### What is `CodeAgent`?  

`CodeAgent` is a type of agent in the `smolagents` library that can generate and execute Python code as part of its reasoning process. This allows agents to perform computations, analyze data, and use external tools in a structured manner.  

### Defining the Experts  

We will create two expert agents:  

- **NLP Expert** – Specializes in natural language processing.  
- **Computer Vision Expert** – Focuses on computer vision tasks.  

Each agent will use an LLM to generate code-based responses when solving domain-specific tasks.

In [5]:
nlp_lecturer_agent = CodeAgent(
    name="nlp_lecturer",
    description="A lecturer in Natural Language Processing (NLP). This agent is a lecturer in NLP and can answer questions about NLP. When not sure about a topic, you can use the duckduckgo search tool to find the answer.",
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10
)

cv_lecturer_agent = CodeAgent(
    name="cv_lecturer",
    description="A lecturer in computer vision. This agent is a lecturer in CV and can answer questions about Computer Vision. When not sure about a topic, you can use the duckduckgo search tool to find the answer.",
    tools=[DuckDuckGoSearchTool()],
    model=model,
    max_steps=10
)


## Defining the Course Director Agent  

Now that we have our expert agents for NLP and Computer Vision, we need an agent to oversee the entire process. The **Course Director** agent will act as the central coordinator, ensuring that the experts collaborate effectively to construct a well-structured lecture outline.  

### Role of the Course Director  

The Course Director is responsible for:  
- **Managing the workflow** – Deciding when and how expert agents contribute.  
- **Gathering insights** – Requesting input from the NLP and Computer Vision experts.  
- **Refining the lecture outline** – Iteratively improving structure and content based on expert feedback.  
- **Ensuring coherence** – Making sure the final outline is clear, logical, and relevant.  

### Implementing the Course Director Agent  

We will define the Course Director as a `CodeAgent`, similar to the experts, but with a broader system prompt to guide decision-making and task delegation.


In [6]:
agent = CodeAgent(
    name='Course_Director',
    description="A course director who manages lecturers.",
    tools=[],
    managed_agents=[cv_lecturer_agent, nlp_lecturer_agent],
    model=model,
    verbosity_level=LogLevel.DEBUG,
)


## Interacting with the Multi-Agent System  

Now that we have defined our agents—the **Course Director**, **NLP Expert**, and **Computer Vision Expert**—it's time to interact with them. You, as the user, will provide an initial topic or high-level prompt, and the agents will collaborate to generate a structured lecture outline.  


In [7]:

prompt = """
You are the course director of a university. You are responsible to coordinate the different topic experts and lecturers and consult them when ever neccessary to get the best possible answer. Always encurage the experts to do background research online prior answering the inquiry.
"Put me together an outline for a course on Generative AI"
"""
answer = agent.run(prompt)

╭─────────────────────────────────────────── New run - Course_Director ───────────────────────────────────────────╮
│                                                                                                                 │
│ You are the course director of a university. You are responsible to coordinate the different topic experts and  │
│ lecturers and consult them when ever neccessary to get the best possible answer. Always encurage the experts to │
│ do background research online prior answering the inquiry.                                                      │
│ "Put me together an outline for a course on Generative AI"                                                      │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
To create a comprehensive course outline on Generative AI, I will proceed by consulting with the NLP and CV        
lecturers. They will be able to provide insights into the important topics that should be covered and suggest      
resources and possible structure ideas for the course. I will encourage them to perform background research to     
ensure the course content is up-to-date.                                                                           
                                                                                                                   
Let's start by consulting the NLP lecturer for their input on generative AI in the context of natural language     
processing.                                                                                                        

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            To create a comprehensive course outline on Generative AI, I will proceed by consulting with the NLP 
and CV lecturers. They will be able to provide insights into the important topics that should be covered and 
suggest resources and possible structure ideas for the course. I will encourage them to perform background research
to ensure the course content is up-to-date.

Let's start by consulting the NLP lecturer for their input on generative AI in the context of natural language 
processing.</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 2.53 seconds| Input tokens: 2,261 | Output tokens: 89]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Now that I have gathered inputs from both the NLP and CV lecturers, it's time to compile a comprehensive  
course outline for Generative AI. This outline will cover both NLP and computer vision aspects, integrating        
suggested resources, learning objectives, and current trends.                                                      
                                                                                                                   
<code>                                                                                                             
# Compiling the course outline using inputs from both NLP and CV lecturers                                         
course_outline = {                                                                                                 
    "title": "Comprehensive Course on Generative AI",                                                              
    "modules": [                                                                                                   
        # NLP Modules                                                                                              
        {                                                                                                          
            "title": "Generative AI in Natural Language Processing",                                               
            "topics": [                                                                                            
                "Fundamentals of NLP & Generative AI",                                                             
                "Large Language Models (LLMs)",                                                                    
                "Generative Adversarial Networks (GANs)",                                                          
                "Applications in NLP",                                                                             
                "Ethical Considerations"                                                                           
            ],                                                                                                     
            "learning_objectives": [                                                                               
                "Gain a solid foundation in NLP and Generative AI.",                                               
 [38;2;

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Compiling the course outline using inputs from both NLP and CV lecturers                                       
  course_outline = {                                                                                               
      "title": "Comprehensive Course on Generative AI",                                                            
      "modules": [                                                                                                 
          # NLP Modules                                                                                            
          {                                                                                                        
              "title": "Generative AI in Natural Language Processing",                                             
              "topics": [                                                                                          
                  "Fundamentals of NLP & Generative AI",                                                           
                  "Large Language Models (LLMs)",                                                                  
                  "Generative Adversarial Networks (GANs)",                                                        
                  "Applications in NLP",                                                                           
                  "Ethical Considerations"                                                                         
              ],                                                                                                   
              "learning_objectives": [                                                                             
                  "Gain a solid foundation in NLP and Generative AI.",                                             
                  "Understand and implement LLMs in practical contexts.",                                          
                  "Develop capabilities to apply GANs for NLP.",                                                   
                  "Address ethical issues in AI system deployment."                                                
              ],                                                                                                   
              "suggested_resources": [                                                                             
                  "Generative AI with LLMs by deeplearning.ai",                                                    
                  "Research papers on arXiv concerning NLP",                                                       
                  "Literature on AI ethics focusing on bias and privacy"                                           
              ]                                                                                                    
          },                                                                                                       
          # CV Modules                                                                                             
          {                                                                                                        
              "title": "Generative AI in Computer Vision",                                                         
              "topics": [                                                                                          
                  "Introduction to Generative AI",                                                                 
                  "Core Techniques and Models",                                                                    
                  "Applications and Case Studies",                                                                 
                  "Ethical and Societal Impacts"         

Final answer: {'title': 'Comprehensive Course on Generative AI', 'modules': [{'title': 'Generative AI in Natural 
Language Processing', 'topics': ['Fundamentals of NLP & Generative AI', 'Large Language Models (LLMs)', 'Generative
Adversarial Networks (GANs)', 'Applications in NLP', 'Ethical Considerations'], 'learning_objectives': ['Gain a 
solid foundation in NLP and Generative AI.', 'Understand and implement LLMs in practical contexts.', 'Develop 
capabilities to apply GANs for NLP.', 'Address ethical issues in AI system deployment.'], 'suggested_resources': 
['Generative AI with LLMs by deeplearning.ai', 'Research papers on arXiv concerning NLP', 'Literature on AI ethics 
focusing on bias and privacy']}, {'title': 'Generative AI in Computer Vision', 'topics': ['Introduction to 
Generative AI', 'Core Techniques and Models', 'Applications and Case Studies', 'Ethical and Societal Impacts'], 
'learning_objectives': ['Understand the key milestones in Generative AI for vision.', 'Explore foundational models 
like SAM and ViTs.', 'Apply generative models to computer vision applications.', 'Critically evaluate ethical 
implications of Generative AI.'], 'suggested_resources': ['Research papers on computer vision models and 
transformers', 'Books/articles on synthetic data generation and GANs', 'Reviews on ethics in AI visual media']}], 
'current_trends': ['Integration of AI models across sectors like healthcare, finance.', 'Growing role of LLMs like 
ChatGPT in various industries.', 'Ongoing ethical debates around AI responsibility and transparency.', 
'Advancements in synthetic data and real-time application methods.']}

[Step 4: Duration 5.35 seconds| Input tokens: 12,654 | Output tokens: 914]

In [8]:
print(answer)

{'title': 'Comprehensive Course on Generative AI', 'modules': [{'title': 'Generative AI in Natural Language Processing', 'topics': ['Fundamentals of NLP & Generative AI', 'Large Language Models (LLMs)', 'Generative Adversarial Networks (GANs)', 'Applications in NLP', 'Ethical Considerations'], 'learning_objectives': ['Gain a solid foundation in NLP and Generative AI.', 'Understand and implement LLMs in practical contexts.', 'Develop capabilities to apply GANs for NLP.', 'Address ethical issues in AI system deployment.'], 'suggested_resources': ['Generative AI with LLMs by deeplearning.ai', 'Research papers on arXiv concerning NLP', 'Literature on AI ethics focusing on bias and privacy']}, {'title': 'Generative AI in Computer Vision', 'topics': ['Introduction to Generative AI', 'Core Techniques and Models', 'Applications and Case Studies', 'Ethical and Societal Impacts'], 'learning_objectives': ['Understand the key milestones in Generative AI for vision.', 'Explore foundational models l

## Congratulations!

You have successfully completed this exercise on building a multi-agent system using `smolagents`. Throughout this notebook, you:  

- Learned how to define and configure expert agents using `CodeAgent`.  
- Created an **NLP Expert** and a **Computer Vision Expert** to provide domain-specific insights.  
- Implemented a **Course Director** agent to coordinate the workflow and manage agent interactions.  
- Interacted with the multi-agent system to generate a structured lecture outline.  
- Observed how agents collaborate, exchange information, and iteratively refine their output.  

This exercise demonstrated how multi-agent systems can be used to automate complex reasoning and content creation tasks. You can extend this framework by adding new expert agents, integrating additional tools, or refining the agents' prompts to improve performance.  
